<strong>Dependencies installation</strong>

In [1]:
%pip install pandas
%pip install numpy
%pip install tensorflow
%pip install scikit-learn
%pip install scikit-metrics
%pip install tensorflow_datasets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


<strong>Dependencies import</strong>

In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras import layers, models # type: ignore



2024-08-15 09:50:10.380013: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 09:50:10.380064: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 09:50:10.381322: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-15 09:50:10.389490: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-15 09:50:11.345320: W tensorflow/comp

In [3]:
# Importazione del dataset

dataset_url = "https://raw.githubusercontent.com/timothypesi/Data-Sets-For-Machine-Learning-/main/california_housing_train.csv"
df = pd.read_csv(dataset_url)

In [4]:
# Suddivisione del dataset in training set e test set

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

In [5]:
# Preparazione dei dati per il modello TensorFlow

train_labels = train_df.pop('median_house_value')
test_labels = test_df.pop('median_house_value')

In [6]:
# Normalizzazione dei dati

normalizer = tf.keras.layers.Normalization()
normalizer.adapt(train_df, steps=200)
normalizer.adapt(test_df, steps=200)


# def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
#   def input_function():  # inner function, this will be returned
#     ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
#     if shuffle:
#       ds = ds.shuffle(1000)  # randomize order of data
#     ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
#     return ds  # return a batch of the dataset
#   return input_function  # return a function object for use

2024-08-15 09:50:20.132132: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [7]:
# Costruzione del modello ANN con TensorFlow

from sklearn.model_selection import learning_curve


model = models.Sequential([
    normalizer,
    layers.Dense(200, activation='relu'),
    layers.Dense(200, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss='mse')

# Addestramento del modello
#model.fit(train_df, train_labels, epochs=200, validation_split=0.2) 

model.fit(train_df, train_labels, epochs=20) 

#notes:
# loss converges at epoch 199, after that increases again
# SGD not working


Epoch 1/20
425/425 [==============================] - 1s 1ms/step - loss: 7113651712.0000
Epoch 2/20
425/425 [==============================] - 1s 2ms/step - loss: 4716696064.0000
Epoch 3/20
425/425 [==============================] - 1s 2ms/step - loss: 4709035520.0000
Epoch 4/20
425/425 [==============================] - 1s 2ms/step - loss: 4627963392.0000
Epoch 5/20
425/425 [==============================] - 1s 2ms/step - loss: 4507360768.0000
Epoch 6/20
425/425 [==============================] - 1s 2ms/step - loss: 4580492800.0000
Epoch 7/20
425/425 [==============================] - 1s 2ms/step - loss: 4439217664.0000
Epoch 8/20
425/425 [==============================] - 1s 2ms/step - loss: 4456439808.0000
Epoch 9/20
425/425 [==============================] - 1s 2ms/step - loss: 4327586816.0000
Epoch 10/20
425/425 [==============================] - 1s 2ms/step - loss: 4175951360.0000
Epoch 11/20
425/425 [==============================] - 1s 2ms/step - loss: 3996518912.0000
Epoch 12

In [8]:
# Valutazione del modello sul test set

loss = model.evaluate(test_df, test_labels)
print(loss)

107/107 [==============================] - 0s 2ms/step - loss: 3136832000.0000
3136832000.0


In [9]:
# Predizione dei valori sul test set

test_predictions = model.predict(test_df).flatten()

107/107 [==============================] - 0s 1ms/step


In [10]:
# Aggiunta delle predizioni al test set

test_df['predicted_median_house_value'] = test_predictions
test_df['median_house_value'] = test_labels


In [11]:
# Calcolo della differenza per record

test_df['value_difference'] = test_df['median_house_value'] - test_df['predicted_median_house_value']

In [12]:
# Salvataggio del modello e dei risultati

model.save("/home/azureuser/cloudfiles/code/Users/g.scorpaniti/deeplearning/ex4_housing_python/artifacts/model/model.keras")
test_df.to_csv("/home/azureuser/cloudfiles/code/Users/g.scorpaniti/deeplearning/ex4_housing_python/artifacts/test_set.csv", index=False)


In [13]:
# Calcolo degli indicatori R2 e MSA

r2 = r2_score(test_labels, test_predictions)
msa = mean_squared_error(test_labels, test_predictions)

print(f"R2 Score: {r2:.4f}")
print(f"Mean Squared Error: {msa:.4f}")

R2 Score: 0.7723
Mean Squared Error: 3136831467.3218
